In [30]:
import cv2
import numpy as np
import json

from object_detector import ObjectDetector

In [31]:
model_config = {
    "efficientdet-lite4": {
        "model_path": "../../models/efficientdet-lite/efficientdet-lite4-detection-default.tflite",
        "input_width": 640,
        "input_height": 640,
        "classes_path": "../../data/coco-efficientdet.names",
        "is_yolo": False
    },
    "efficientdet-lite3": {
        "model_path": "../../models/efficientdet-lite/efficientdet-lite3-detection-default.tflite",
        "input_width": 512,
        "input_height": 512,
        "classes_path": "../../data/coco-efficientdet.names",
        "is_yolo": False
    },
    "efficientdet-lite2": {
        "model_path": "../../models/efficientdet-lite/efficientdet-lite2-detection-default.tflite",
        "input_width": 448,
        "input_height": 448,
        "classes_path": "../../data/coco-efficientdet.names",
        "is_yolo": False
    },
    "efficientdet-lite1": {
        "model_path": "../../models/efficientdet-lite/efficientdet-lite1-detection-default.tflite",
        "input_width": 384,
        "input_height": 384,
        "classes_path": "../../data/coco-efficientdet.names",
        "is_yolo": False
    },
    "ssd-mobilenet-v1": {
        "model_path": "../../models/mobilenetv1/ssd-mobilenet-v1-default.tflite",
        "input_width": 300,
        "input_height": 300,
        "classes_path": "../../data/coco-efficientdet.names",
        "is_yolo": False
    },
    "yolov5-small": {
        "model_path": "../../models/yolov5-small/yolov5-small.tflite",
        "input_width": 320,
        "input_height": 320,
        "classes_path": "../../data/coco.names",
        "is_yolo": True
    },
    "yolov5-nano": {
        "model_path": "../../models/yolov5/yolov5n-int8.tflite",
        "input_width": 320,
        "input_height": 320,
        "classes_path": "../../data/coco.names",
        "is_yolo": True
    }
}

CURRENT_MODEL = model_config["yolov5-nano"]

object_detector = ObjectDetector(
    model_path=CURRENT_MODEL["model_path"],
    input_width=CURRENT_MODEL["input_width"],
    input_height=CURRENT_MODEL["input_height"],
    classes_path=CURRENT_MODEL["classes_path"],
    is_yolo=CURRENT_MODEL["is_yolo"]
)

In [32]:
with open('Objects365/test/filtered_data_coco.json', 'r') as f:
    ds = json.load(f)

# Store COCO detection results
detection_results = []

ann_id = 0

total_processing_time = 0

# filter specific image
# ds["images"] = [d for d in ds["images"] if d["id"] == 684819]

for i, img in enumerate(ds["images"]):
    # Read the frame and resize it
    frame = cv2.imread(f"./Objects365/test/full-ds/{img['file_name']}")
    frame_height, frame_width, _ = frame.shape
    frame_resized = cv2.resize(frame, (CURRENT_MODEL["input_width"], CURRENT_MODEL["input_height"]))
    input_data = np.expand_dims(frame_resized, axis=0)
    if CURRENT_MODEL["is_yolo"]:
        input_data = np.expand_dims(frame_resized, axis=0).astype(np.float32) / 255.0

    detected_objects, processing_time = object_detector.process_frame(frame=input_data, frame_width=frame_width, frame_height=frame_height)
    total_processing_time += processing_time

    # Store the detection results
    for obj in detected_objects:
        ann_id += 1
        detection_results.append({
            "image_id": img["id"],
            "category_id": int(obj.category_id),
            "bbox": list(obj.bbox),
            "score": float(obj.confidence)
        })
        
    if i % 100 == 0:
        print(f"{i}/{len(ds['images'])}: {total_processing_time / (i + 1)}, {1 / (total_processing_time / (i + 1))}")

    # annotated_frame = object_detector.draw_frame_bounding_boxes(frame, detected_objects)

    # cv2.imshow('Object Detection', annotated_frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# Save the detection results
with open('./Objects365/test/yolov5n-int8.json', 'w') as f:
    json.dump(detection_results, f)

ValueError: Cannot set tensor: Dimension mismatch. Got 320 but expected 3 for dimension 1 of input 0.